### TensorFlow : theano에 비해 분산처리기능 강화
1. 심볼 변수 정의
2. 심볼 관계 정의
3. 세션 정의
4. 세션 사용

##### 1. 심볼 변수 정의

In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

In [3]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

##### 2. 심볼 관계 정의

In [4]:
z = x + y
type(z)

tensorflow.python.framework.ops.Tensor

In [5]:
u = tf.log(z)
type(u)

tensorflow.python.framework.ops.Tensor

##### 3. Session 정의 및 사용
- theano의 함수와 유사한 역할
- run() : 그래프의 값을 실제로 계산하고 결과를 반환
- close() : with문으로 대체하기도 한다.

In [6]:
sess = tf.Session()
init = tf.global_variables_initializer() # 변수 초기값 대입을 위한 특별 그래프
sess.run(init)
print(sess.run(z))
print(sess.run(u))
sess.close()

3.0
1.0986123


##### 미분

In [7]:
f = x**2
fx = tf.gradients(f,[x])

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))
    print(sess.run(fx))

1.0
[2.0]


In [8]:
!rm -rf C:/home/dockeruser/logs/*

'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


##### 퍼셉트론 제작
- iris데이터 중 두 종류를 분류하는 퍼셉트론 제작
- y는 -1 or 1을 사용하고 , 활성화함수로 hypertangent를 사용
- GradientDescentOptimizer() 등의 클래스를 통해 update그래프가 튀어나온다.

In [9]:
from sklearn.datasets import load_iris
iris = load_iris()

In [27]:
idx = np.in1d(iris.target, [0,2])
x = iris.data[idx, 0:2]
y = (iris.target[idx] - 1.0)[:,np.newaxis] # 1.0으로 빼줘야함, 2차원으로 바꿔줘야함.

In [28]:
np.random.seed(0)
w = tf.Variable(1e-3 * np.random.randn(2,1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])
z = tf.tanh(tf.matmul(x,w) + b)

In [29]:
zero = np.zeros(1, dtype=np.float64)[0]
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y,z)))
optimizer = tf.train.GradientDescentOptimizer(1e-7) # learning rate
train = optimizer.minimize(cost)

In [30]:
sess = tf.Session()

w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.scalar("b",b)
z_summary = tf.summary.histogram("z",z)
cost_summary = tf.summary.scalar("cost", cost)
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter("C:/home/dockeruser/logs", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

for i in range(100):
    sess.run(train)
    summary = sess.run(merged)
    writer.add_summary(summary, i)
    if i%10==0:
        print("{:4d}: ".format(i), sess.run(cost))

   0:  0.5494091733308987
  10:  0.4550718147888182
  20:  0.3607278431087564
  30:  0.2663786243415365
  40:  0.17202552496054824
  50:  0.07766991175203046
  60:  0.0009515019028440593
  70:  0.0003168280919044534
  80:  0.0002921058755255818
  90:  0.00029228987473695566


In [31]:
y_pred = sess.run(tf.sign(z))

In [32]:
sess.close()

In [33]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 1, 49]], dtype=int64)